In [1]:
import sys
import os
import numpy as np

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

from src import data_processing
from src.data_processing import load_csv,drop_columns,fill_missing_mode,standard,encode_label,write_csv,fill_missing_constant,encode_columns_with_mapper


In [2]:
data,col_names = load_csv('../data/raw/aug_train.csv')
data_test,col_names_test = load_csv('../data/raw/aug_test.csv')

## Data cleaning

Đầu tiên ta xử lí missing values
- Như đã khám phá ở trước các cột `gender`, `company_type`, `company_size`, `major_discipline` sẽ điền thêm giá trị unknown
- Các cột còn lại sẽ điền mode

In [3]:
cols_unknown = ['gender', 'company_type', 'company_size', 'major_discipline']
idx_unknown = [col_names.index(c) for c in cols_unknown]
data = fill_missing_constant(data,idx_unknown)

cols_mode = ['enrolled_university', 'education_level', 'last_new_job', 'experience']
idx_mode = [col_names.index(c) for c in cols_mode]
data = fill_missing_mode(data,idx_mode)


In [4]:
print("=== BẢNG TỔNG HỢP MISSING VALUES ===")
print(f"{'Tên cột':<25}|{'Số lượng':>10}|")
print("-" * 37)


for idx, name in enumerate(col_names):
    col = data[:,idx]
    count = np.sum(col == '')
    print(f"{name:<25}|{count:>10d}|")


=== BẢNG TỔNG HỢP MISSING VALUES ===
Tên cột                  |  Số lượng|
-------------------------------------
enrollee_id              |         0|
city                     |         0|
city_development_index   |         0|
gender                   |         0|
relevent_experience      |         0|
enrolled_university      |         0|
education_level          |         0|
major_discipline         |         0|
experience               |         0|
company_size             |         0|
company_type             |         0|
last_new_job             |         0|
training_hours           |         0|
target                   |         0|


Tương tự với `test`

In [5]:
data_test = fill_missing_constant(data_test,idx_unknown)
data_test = fill_missing_mode(data_test,idx_mode)
print("=== BẢNG TỔNG HỢP MISSING VALUES ===")
print(f"{'Tên cột':<25}|{'Số lượng':>10}|")
print("-" * 37)


for idx, name in enumerate(col_names_test):
    col = data_test[:,idx]
    count = np.sum(col == '')
    print(f"{name:<25}|{count:>10d}|")

=== BẢNG TỔNG HỢP MISSING VALUES ===
Tên cột                  |  Số lượng|
-------------------------------------
enrollee_id              |         0|
city                     |         0|
city_development_index   |         0|
gender                   |         0|
relevent_experience      |         0|
enrolled_university      |         0|
education_level          |         0|
major_discipline         |         0|
experience               |         0|
company_size             |         0|
company_type             |         0|
last_new_job             |         0|
training_hours           |         0|


## Feature Engineering

Ta cần gom nhóm các city lại vì có quá nhiều city

In [6]:
idx_city = col_names.index('city')
cty = data[:,idx_city]
unique_cty , count_cty = np.unique(cty,return_counts=True)
sorted_indices = np.argsort(-count_cty)
top_cities = unique_cty[sorted_indices][:10] 
mask_other = ~np.isin(cty,top_cities)
cty[mask_other] = 'Other'
data[:,idx_city] = cty

cty = data_test[:,idx_city]
mask_other = ~np.isin(cty,top_cities)
cty[mask_other] = 'Other'
data_test[:,idx_city] = cty

Như đã trình bày trong khám phá dữ liệu thì việc ở thành phố thấp có tỉ lệ đổi việc khá cao nên ta sẽ thêm 1 feature vô đây

In [7]:
idx_cdi = col_names.index('city_development_index')
col_cdi = data[:, idx_cdi].astype(float)
feature_low_cdi = np.where(col_cdi < 0.65, '1', '0')
insert_position = data.shape[1] - 1  
data = np.insert(data, insert_position, feature_low_cdi, axis=1)
col_names.insert(insert_position, 'is_low_cdi')

col_cdi = data_test[:, idx_cdi].astype(float)
feature_low_cdi = np.where(col_cdi < 0.65, '1', '0')
data_test = np.column_stack((data_test, feature_low_cdi))
col_names_test.append('is_low_cdi')

## Feature Encoding

Ta sẽ tiến hành encoding dựa theo danh sách các giá trị unique của từng cột `categorial`
Với các cột quan trọng thứ tự ta cần định nghĩa mapping

In [8]:
encode = {}
encode[col_names.index('experience')] = {
    '<1': 0, 
    '>20': 21, 
    'Unknown': -1,
}
for i in range(1,21):
    encode[col_names.index('experience')][str(i)] = i
encode[col_names.index('company_size')] = {
    'Unknown': -1, 
    '<10': 0, 
    '10/49': 1, 
    '50-99': 2, 
    '100-500': 3, 
    '500-999': 4, 
    '1000-4999': 5, 
    '5000-9999': 6, 
    '10000+': 7
}
encode[col_names.index('education_level')] = {
    'Unknown': -1, 
    'Primary School': 0, 
    'High School': 1, 
    'Graduate': 2, 
    'Masters': 3, 
    'Phd': 4
}

encode[col_names.index('last_new_job')] = {
    'Unknown': -1, 
    'never': 0, 
    '1': 1, 
    '2': 2, 
    '3': 3, 
    '4': 4, 
    '>4': 5
}
encode[col_names.index('enrolled_university')] = {
    'Unknown': -1, 
    'no_enrollment': 0, 
    'Part time course': 1, 
    'Full time course': 2
}


In [9]:
cat_col1 = [col_names.index(name) for name in  ['experience','company_size','education_level','last_new_job','enrolled_university']]
data = encode_columns_with_mapper(data,cat_col1,encode)

Với các categorial còn lại thì ta chỉ việc điền encode theo dict

In [10]:
cat_col2 = [col_names.index(name) for name in  ['gender', 'company_type', 'city', 'major_discipline','enrollee_id','relevent_experience']]
data,tmp = encode_label(data,cat_col2)

In [11]:
encode.update(tmp)
cat_col = [i for i in range(len(col_names_test)) if col_names_test[i] not in ['enrollee_id','city_development_index','training_hours','is_low_cdi']]
data_test = encode_columns_with_mapper(data_test,cat_col,encode)


## Feature Selection

Ta sẽ bỏ các cột `enrollee_id` vì cột này chỉ là id

In [12]:
cols_to_drop = ['enrollee_id']
#,'education_level' ,'last_new_job','gender','major_discipline','training_hours'
data, col_names = drop_columns(col_names,data, cols_to_drop)

print(f"Đã xóa các cột: {cols_to_drop}")
print(f"Số cột còn lại: {len(col_names)}")
print(f"Danh sách cột mới: {col_names}")

Đã xóa các cột: ['enrollee_id']
Số cột còn lại: 14
Danh sách cột mới: ['city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'is_low_cdi', 'target']


In [13]:
data_test, col_names_test = drop_columns(col_names_test,data_test, cols_to_drop)

print(f"Đã xóa các cột: {cols_to_drop}")
print(f"Số cột còn lại: {len(col_names_test)}")
print(f"Danh sách cột mới: {col_names_test}")

Đã xóa các cột: ['enrollee_id']
Số cột còn lại: 13
Danh sách cột mới: ['city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'is_low_cdi']


## Feature Scaling

In [14]:
data[:,:-1],mean,std = standard(data[:,:-1])
data_test,_,_ = standard(data_test,mean,std)

Sau khi hoàn tất khi lại vô file

In [15]:
write_csv(data,col_names,'../data/processed/aug_train.csv')
write_csv(data_test,col_names_test,'../data/processed/aug_test.csv')